In [1]:
import numpy as np
import pandas as pd

from compoundDB import inputtools as it
from UpdateDB_ref import Checkpoint as cp
from UpdateDB_ref import CR
from UpdateDB_ref import Update_CII as updater


*** CompoundDB module found. Will check the synonyms table to resolve CAS. ***



#### Load list into pandas

In [2]:
example_df = pd.read_csv('example.csv', sep=',')

#### Processing example dataframe
First we get CAS and then we generate a list with unique CAS. Finally we'll get the annotations for each CAS from CR database.
Is important to remember that each input should be curated manually to end up having the same format obtained at the end with example_annotations dataframe.

In [3]:
example_df.loc[:,'N.o CAS'] = example_df['N.o CAS'].str.split('/')
example_df.loc[:,'N.o CAS'] = example_df['N.o CAS'].astype(str).apply(lambda x: x.strip('[').strip(']').replace("'","").strip().replace(' and several other',''))

In [4]:
example_copy = example_df.copy()

In [5]:
new_data = {'Sustancia':[],'Listado individual':[], 'N.o CAS':[]}
for i, row in example_df.iterrows():
    subs_row = row['Sustancia']
    pref_name = row['Listado individual']
    cas_row = row['N.o CAS']
    if ',' in cas_row:
        cas_list = cas_row.split(',')
        for cas in cas_list:
            new_data['Sustancia'].append(subs_row)
            new_data['Listado individual'].append(pref_name)
            new_data['N.o CAS'].append(cas.strip())
        example_copy.drop(i, axis=0, inplace=True)
new_df = pd.DataFrame(new_data)

In [6]:
example_conc = pd.concat([example_copy, new_df])
example_conc.reset_index(inplace=True)
example_conc.drop(labels='index',axis=1, inplace=True)
example_conc.rename(columns={'N.o CAS':'CAS'},inplace=True)

#### Connection to CR
I connect to CR and extract annotations for each CAS

In [7]:
ann_df = CR.CR().get_annotations_per_CAS()

In [8]:
ann_df.loc[ann_df['type'] == 'Negative', 'original_annotation'] = ann_df.loc[ann_df['type'] == 'Negative', 'original_annotation'].apply(lambda x: ' '.join(['Not',x]))

In [9]:
ann_df.loc[ann_df['type'] == 'Negative'].head()

,CAS,source_name,original_annotation,type,annotation,general,category
13,100-00-5,REACH Annex VI,Not PBT,Negative,PBT,PBT,PBT
8,100-00-5,REACH Annex VI,Not vPvB,Negative,vPvB,,vPvB
101,100-01-6,REACH Annex VI,Not Carcinogenicity,Negative,Carcinogenic,Carcinogen,CMR
106,100-01-6,REACH Annex VI,Not Mutagenicity,Negative,Mutagenic,Mutagen,CMR
108,100-01-6,REACH Annex VI,Not PBT,Negative,PBT,PBT,PBT


#### Preparing dataframes with annotations
Here I create a new dataframe with CR annotations for each CAS in LSR. If I use pd.merge with how='inner' it allows me to recover all CAS from LSR dataframe avoiding empty substances

In [10]:
example_annotations = example_conc.merge(ann_df, on='CAS', how='inner')

In [11]:
example_annotations.head()

,Sustancia,Listado individual,CAS,source_name,original_annotation,type,annotation,general,category
0,Arilaminas,4-Aminobiphenyl,92-67-1,CLP Notification,Carc. 1A,Confirmed,Carc. 1A,Carcinogen,CMR
1,Arilaminas,4-Aminobiphenyl,92-67-1,CLP Notification,Carc. 1B,Confirmed,Carc. 1B,Carcinogen,CMR
2,Arilaminas,4-Aminobiphenyl,92-67-1,CLP Notification,Muta. 1B,Confirmed,Muta. 1B,Mutagen,CMR
3,Arilaminas,4-Aminobiphenyl,92-67-1,CLP Notification,Muta. 2,Confirmed,Muta. 2,Mutagen,CMR
4,Arilaminas,4-Aminobiphenyl,92-67-1,EPA Genetox,Sister-chromatid exchange (SCE) in vivo,Confirmed,Sister-chromatid exchange (SCE) in vivo,Genotoxic,CMR


#### Add dataframe to devel CII
Add new substances to CII or update the ones that are already there. I use original_annotation because it includes EUH annotations, and annotation column doesn't.

In [12]:
updater = updater.UpdateDB(host='localhost', dbname='cii_test', user='postgres', password='PSGAdmin')

Add all the information from the dataframe: substance, chemical identifiers, structure, sources, annotations and how they're related (regulations table)

In [13]:
updater.add_all_information_from_dataframe(dataframe=example_annotations,
                                    class_name_field = 'Sustancia',
                                    preferred_name_field = 'Listado individual',
                                    chem_id_field='CAS',
                                    chem_id_type='casr_number',
                                    sourceName_field='source_name',
                                    regulation_field='original_annotation')

Add only substances by name

In [14]:
updater.add_substances_from_dataframe(dataframe=example_annotations,
                                    class_name_field = 'Sustancia',
                                    preferred_name_field = 'Listado individual')

Add chemical identifiers and substances by name

In [15]:
updater.add_chemical_identifier_from_dataframe(dataframe=example_annotations,
                                            class_name_field = 'Sustancia',
                                            preferred_name_field = 'Listado individual',
                                            chem_id_field='CAS',
                                            chem_id_type='casr_number')

Add only structures (SMILES) if present in the dataframe. If not, it generates the structure from the CAS and stores it

In [16]:
small_strucs = updater.get_substances_with_structure()[:10]

In [17]:
small_strucs

,class_name_curated,preferred_name_curated,name,chem_id,structure,structure_curated,substance_type_id
0,Formaldehyde,Methanediol,463-57-0,2,OCO,OCO,1.0
1,Formaldehyde,Formaldehyde,50-00-0,1,C=O,C=O,1.0
2,"N,N-Dimethylformamide","N,N-Dimethylformamide",68-12-2,5,CN(C)C=O,CN(C)C=O,1.0
3,None,Quinoline,91-22-5,7,c1ccc2ncccc2c1,c1ccc2ncccc2c1,1.0
4,N-nitrosamines,N-Nitrosodimethylamine,62-75-9,9,CN(C)N=O,CN(C)N=O,1.0
5,N-nitrosamines,N-Nitrosodiethylamine,55-18-5,11,CCN(CC)N=O,CCN(CC)N=O,1.0
6,N-nitrosamines,N-Nitrosodipropylamine,621-64-7,13,CCCN(CCC)N=O,CCCN(CCC)N=O,1.0
7,N-nitrosamines,N-Nitrosodibutylamine,924-16-3,15,CCCCN(CCCC)N=O,CCCCN(CCCC)N=O,1.0
8,N-nitrosamines,N-Nitrosopiperidine,100-75-4,17,O=NN1CCCCC1,O=NN1CCCCC1,1.0
9,N-nitrosamines,N-Nitrosopyrrolidine,930-55-2,19,O=NN1CCCC1,O=NN1CCCC1,1.0


In [18]:
updater.add_structure_from_dataframe(dataframe=small_strucs,
                                    class_name_field = 'class_name_curated',
                                    preferred_name_field = 'preferred_name_curated',
                                    chem_id_field='name',
                                    chem_id_type='casr_number',
                                    smiles_field='structure')

Add only the sources present in the dataframe

In [19]:
updater.add_source_from_dataframe(dataframe=example_annotations,
                                 sourceName_field='source_name')

Add only the annotations (hazards etc...) present in the dataframe

In [20]:
updater.add_annotation_from_dataframe(dataframe=example_annotations,
                                     annotation_field='original_annotation')